<hr>
<h1><center>Handling missing values</center></h1>
<hr>

In this post, we'll review some basic methods to handle missing values in our datasets

IMPORTANT: This is just a recopilation of simple methods. The "best" method, or the optimal method, must be selected according to the problem we are facing. Or we can evaluate several approaches.

#### Outline

* [Basic approaches using pandas](#first-bullet)
* [Using skit-learn](#second-bullet)
    * [SimpleImputer class](#2.1)
* [State of the art methods](#third-bullet)
* [References](#fourth)

Let's start by creating a toy dataset

In [1]:
import numpy as np

Glucosa = np.random.normal(0,0.1, size=(12))
Glucosa[0]=Glucosa[4]=Glucosa[7]=np.nan

In [2]:
boxes = {
'age':[24,np.nan,35,42,np.nan,28,29,19,32,33,28,25],
'gender':[0,1,1,0,1,np.nan,0,0, 1,1,0,1],
'Glucosa': Glucosa,
'Respiratory_rate':np.random.uniform(low=0, high=1, size=(12)),
}

In [3]:
import pandas as pd

data = pd.DataFrame(boxes)
data

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
1,NaN,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
4,NaN,1.0,NaN,0.359133
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248


## 1. Basic approaches using pandas <a class="anchor" id="first-bullet"></a>

### Reviewing the missing values

In [4]:
data.isnull()

,age,gender,Glucosa,Respiratory_rate
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,False,False,False,False
4,True,False,True,False
5,False,True,False,False
6,False,False,False,False
7,False,False,True,False
8,False,False,False,False
9,False,False,False,False


In [5]:
data.isna()

,age,gender,Glucosa,Respiratory_rate
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,False,False,False,False
4,True,False,True,False
5,False,True,False,False
6,False,False,False,False
7,False,False,True,False
8,False,False,False,False
9,False,False,False,False


### Identifying the column(s) which have at least one missing value

In [6]:
print(data.isna().any())

age                  True
gender               True
Glucosa              True
Respiratory_rate    False
dtype: bool


In [7]:
print(data.isnull().any())

age                  True
gender               True
Glucosa              True
Respiratory_rate    False
dtype: bool


### Counting the number of missing values

In [8]:
data.isnull().sum()

age                 2
gender              1
Glucosa             3
Respiratory_rate    0
dtype: int64

In [9]:
data.isna().sum()

age                 2
gender              1
Glucosa             3
Respiratory_rate    0
dtype: int64

In [10]:
data.isna().sum().sum()

6

### Drop the rows where at least one element is missing in a given DataFrame

In [11]:
data_1 = data.copy()

In [12]:
data_1.dropna(axis=0)

,age,gender,Glucosa,Respiratory_rate
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
6,29.0,0.0,0.082293,0.939527
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248
10,28.0,0.0,0.160996,0.822472
11,25.0,1.0,-0.030212,0.453733


In [13]:
data_1.dropna(axis='rows')

,age,gender,Glucosa,Respiratory_rate
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
6,29.0,0.0,0.082293,0.939527
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248
10,28.0,0.0,0.160996,0.822472
11,25.0,1.0,-0.030212,0.453733


### Drop the rows where at least one element is missing in a given DataFrame

In [14]:
data_1 = data.copy()
data_1.dropna(axis=1)

,Respiratory_rate
0,0.337012
1,0.338481
2,0.344331
3,0.592843
4,0.359133
5,0.990947
6,0.939527
7,0.830164
8,0.850873
9,0.854248


In [15]:
data_1.dropna(axis='columns')

,Respiratory_rate
0,0.337012
1,0.338481
2,0.344331
3,0.592843
4,0.359133
5,0.990947
6,0.939527
7,0.830164
8,0.850873
9,0.854248


### Drop the rows in which there are two missing values

In [16]:
data_1.dropna(axis=0, thresh=3)

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
1,NaN,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248
10,28.0,0.0,0.160996,0.822472


### Drop those rows from a given DataFrame in which specific columns have missing values

In [17]:
data_1.dropna(subset=['age'])

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248
10,28.0,0.0,0.160996,0.822472
11,25.0,1.0,-0.030212,0.453733


### Keep the valid entries of a given DataFrame.

In [18]:
data.dropna(inplace=False)

,age,gender,Glucosa,Respiratory_rate
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
6,29.0,0.0,0.082293,0.939527
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248
10,28.0,0.0,0.160996,0.822472
11,25.0,1.0,-0.030212,0.453733


### Replace NaNs with a single constant value in specified columns

In [19]:
data_2 = data['age'].fillna(0, inplace=False)
data_2

0     24.0
1      0.0
2     35.0
3     42.0
4      0.0
5     28.0
6     29.0
7     19.0
8     32.0
9     33.0
10    28.0
11    25.0
Name: age, dtype: float64

### Replace NaNs with the mean of a specific column

In [20]:
data['age'].mean()

29.5

In [21]:
data_3 = data.copy()

data_3['age'].fillna(data_3['age'].mean(), inplace=True)
data_3

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
1,29.5,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
4,29.5,1.0,NaN,0.359133
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248


### Replace NaNs with the value from the previous row 

In [22]:
data_4 = data.copy()

data_4['age'].fillna(method='pad', inplace=True)
data_4

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
1,24.0,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
4,42.0,1.0,NaN,0.359133
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248


### Replace NaNs with the value from the following row 

In [23]:
data_5 = data.copy()

data_5['age'].fillna(method='bfill', inplace=True)
data_5

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
1,35.0,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
4,28.0,1.0,NaN,0.359133
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248


### Replace NaNs with median or mean of the specified columns in a given DataFrame

In [24]:
data_5 = data.copy()
data_5['age'].fillna(data_5['age'].median(), inplace=True)
data_5

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
1,28.5,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
4,28.5,1.0,NaN,0.359133
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248


### Interpolate the missing values using the Linear Interpolation method

In [25]:
data_5 = data.copy()
data['age'].interpolate(method='linear', direction = 'forward', inplace=True) 
data

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
1,29.5,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
4,35.0,1.0,NaN,0.359133
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248


### Replace the missing values with the most frequent values present in each column 

In [26]:
data['age'].mode()

0    28.0
1    35.0
Name: age, dtype: float64

In [27]:
data_5 = data.copy()
data['age'].fillna(data['age'].mode().iloc[0])

0     24.0
1     29.5
2     35.0
3     42.0
4     35.0
5     28.0
6     29.0
7     19.0
8     32.0
9     33.0
10    28.0
11    25.0
Name: age, dtype: float64

## 2. Using sklearn <a class="anchor" id="second-bullet"></a>

In [28]:
data

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
1,29.5,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
4,35.0,1.0,NaN,0.359133
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248


### 2.1 SimpleImputer class <a class="anchor" id="2.1"></a>

The SimpleImputer class provides basic strategies for imputing missing values. 

In [29]:
from sklearn.impute import SimpleImputer

In [30]:
my_imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #strategies = mean, mode, and most_frequent

In [31]:
X = pd.DataFrame(my_imputer.fit_transform(data))


# Imputation removed column names; put them back
X.columns = data.columns
X


# Note the 5th row in the label age

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.000000,0.023982,0.337012
1,29.5,1.000000,-0.116118,0.338481
2,35.0,1.000000,0.044027,0.344331
3,42.0,0.000000,0.034972,0.592843
4,35.0,1.000000,0.023982,0.359133
5,28.0,0.545455,-0.022143,0.990947
6,29.0,0.000000,0.082293,0.939527
7,19.0,0.000000,0.023982,0.830164
8,32.0,1.000000,0.043326,0.850873
9,33.0,1.000000,0.018702,0.854248


### 2.2 Nearest neighbors imputation <a class="anchor" id="2.2"></a>

In [32]:
data

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,NaN,0.337012
1,29.5,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
4,35.0,1.0,NaN,0.359133
5,28.0,NaN,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,NaN,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248


In [33]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform", metric='nan_euclidean')
X_nn =pd.DataFrame(imputer.fit_transform(data))

In [34]:
X_nn.columns=data.columns  # Imputation removed column names; put them back
X_nn  

,age,gender,Glucosa,Respiratory_rate
0,24.0,0.0,0.014963,0.337012
1,29.5,1.0,-0.116118,0.338481
2,35.0,1.0,0.044027,0.344331
3,42.0,0.0,0.034972,0.592843
4,35.0,1.0,0.014446,0.359133
5,28.0,0.6,-0.022143,0.990947
6,29.0,0.0,0.082293,0.939527
7,19.0,0.0,0.014963,0.830164
8,32.0,1.0,0.043326,0.850873
9,33.0,1.0,0.018702,0.854248


## 3. State of the art methods <a class="anchor" id="third-bullet"></a>

Several approaches have been proposed to handle missing values.

* A comparison of imputation methods can be found in [2] and [3].

* Imputation methods, from a optimization approach can be found in [4]. See Table 1: List of Imputation Methods as well.

## 4. References <a class="anchor" id="fourth"></a>

* [1] Scikit Learn. Imputation of missing values  [https://scikit-learn.org/stable/modules/impute.html](https://scikit-learn.org/stable/modules/impute.html) 

* [2] Shrive, F.M., Stuart, H., Quan, H. et al. Dealing with missing data in a multi-question depression scale: a comparison of imputation methods. BMC Med Res Methodol 6, 57 (2006). [https://bmcmedresmethodol.biomedcentral.com/articles/10.1186/1471-2288-6-57](https://bmcmedresmethodol.biomedcentral.com/articles/10.1186/1471-2288-6-57)

* [3] Huque, M.H., Carlin, J.B., Simpson, J.A. et al. A comparison of multiple imputation methods for missing data in longitudinal studies. BMC Med Res Methodol 18, 168 (2018).  [https://bmcmedresmethodol.biomedcentral.com/articles/10.1186/s12874-018-0615-6](https://bmcmedresmethodol.biomedcentral.com/articles/10.1186/s12874-018-0615-6)

* [4] Dimitris Bertsimas, Colin Pawlowski, Ying Daisy Zhuo. From Predictive Methods to Missing Data Imputation: An Optimization Approach.  [https://jmlr.org/papers/v18/17-073.html](https://jmlr.org/papers/v18/17-073.html)